# Downloading Data from Census API for General Domain

I wanted to use the package "censusdis" and/or the package "census" to download census data 
using the API, but we don't have access to it in EDL. I asked Brock Webb about it.

Brock said, "That can be done via a remedy request to install the package. 
The bad thing is... even if you do that, I didn't think EDL had open Internet access, 
so it doesn't work. That was a struggle I had with the python packages. They made it 
very easy to get the data as demonstrated today. However, I couldn't do that, so I 
had to get it manually from data.census.gov and figure out how to get what I wanted 
into a table so I could import the file. "

So instead of using a package, I'm using the following code.

If there is time, these are the things I would add to the code:

1. Where 1-year data are available, use 1-year instead of 5-year (applies to county and tribal areas)
2. Add block group data
3. Add more "loops" and "if-else" code so I don't need as many functions
4. Make adjustments for geo changes over time
5. Make adjustments for Puerto Rico data
6. Make adjustments for tribal area name
7. Additional QC

In [1]:
# Setting up libraries
import requests
import pandas as pd

#Create a blank dataframe for appending data
general_measures = pd.DataFrame()

#create base column names for every geography and year
base_col_names = ['Geo_name', 'Total_population', 'Total_population_moe', 
                 "Median_age", "Median_age_moe",
                 "Percent_non_Hispanic_White_alone", "Percent_non_Hispanic_White_alone_moe",
                 "Percent_non_Hispanic_Black_alone", "Percent_non_Hispanic_Black_alone_moe",
                 "Percent_non_Hispanic_AIAN_alone", "Percent_non_Hispanic_AIAN_alone_moe",
                 "Percent_non_Hispanic_Asian_alone", "Percent_non_Hispanic_Asian_alone_moe",
                 "Percent_non_Hispanic_NHPI_alone", "Percent_non_Hispanic_NHPI_alone_moe",
                 "Percent_non_Hispanic_SOR_alone", "Percent_non_Hispanic_SOR_alone_moe",
                 "Percent_non_Hispanic_Multi_alone", "Percent_non_Hispanic_Multi_alone_moe",
                 "Percent_Hispanic", "Percent_Hispanic_moe",
                 "Percent_bachelors_degree_or_higher", "Percent_bachelors_degree_or_higher_moe",
                 "Percent_high_school_graduate_or_higher", "Percent_high_school_graduate_or_higher_moe",
                 "Median_household_income", "Median_household_income_moe",
                 "Poverty_rate", "Poverty_rate_moe",
                 "Percent_employed", "Percent_employed_moe",
                 "Percent_unemployed", "Percent_unemployed_moe",
                 "Percent_speak_English_less_than_very_well", "Percent_speak_English_less_than_very_well_moe",
                 "Percent_civilian_veteran", "Percent_civilian_veteran_moe",
                 "Percent_no_health_insurance", "Percent_no_health_insurance_moe"]


In [2]:
#This is the function for the nation

def api_us(year, dataset, geo_for, area_type, geo_id, get_vars):
    global general_measures

    # Build base URL
    HOST = "https://api.census.gov/data"
    base_url = "/".join([HOST, year, dataset])

    # Specify Census variables and other predicates
    predicates = {}
    predicates["get"] = ",".join(get_vars)
    predicates["for"] = geo_for

    # Execute the request, examine text of response object
    r = requests.get(base_url, params=predicates)

    # Construct the DataFrame
    col_names = base_col_names + ['GEO_ID','fips']
    df = pd.DataFrame(columns = col_names, data = r.json()[1:])
                
    # Create additional variables for file
    df["year"] = year
    df["area_type"] = area_type
    df["state_fips"] = ""
    df["county_fips"] = ""
    df["tract_fips"] = ""
    df["GEO_ID"] = geo_id  
    df.drop('fips', axis=1, inplace=True)
   
    #append data
    general_measures = pd.concat([general_measures,df], ignore_index=True)

In [3]:
#nation
api_us(year = "2022", dataset = "acs/acs1/profile?", geo_for = "us:*", area_type = "national", geo_id = "us", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0079PE", "DP05_0079PM", #white
                "DP05_0080PE", "DP05_0080PM", #black
                "DP05_0081PE", "DP05_0081PM", #AIAN
                "DP05_0082PE", "DP05_0082PM", #Asian
                "DP05_0083PE", "DP05_0083PM", #NHPI    
                "DP05_0084PE", "DP05_0084PM", #SOR   
                "DP05_0085PE", "DP05_0085PM", #Multi   
                "DP05_0073PE", "DP05_0073PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_us(year = "2021", dataset = "acs/acs1/profile?", geo_for = "us:*", area_type="national", geo_id = "us", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

#1-year data are unavailable for 2020
api_us(year = "2020", dataset = "acs/acs5/profile?", geo_for = "us:*", area_type="national", geo_id = "us", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_us(year = "2019", dataset = "acs/acs1/profile?", geo_for = "us:*", area_type="national", geo_id = "us", get_vars =  
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0114PE", "DP02_0114PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_us(year = "2018", dataset = "acs/acs1/profile?", geo_for = "us:*", area_type="national", geo_id = "us", get_vars =  
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_us(year = "2017", dataset = "acs/acs1/profile?", geo_for = "us:*", area_type="national", geo_id = "us", get_vars =  
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_us(year = "2016", dataset = "acs/acs1/profile?", geo_for = "us:*", area_type="national", geo_id = "us", get_vars =  
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0017E", "DP05_0017M", #median age
                "DP05_0072PE", "DP05_0072PM", #white
                "DP05_0073PE", "DP05_0073PM", #black
                "DP05_0074PE", "DP05_0074PM", #AIAN
                "DP05_0075PE", "DP05_0075PM", #Asian
                "DP05_0076PE", "DP05_0076PM", #NHPI    
                "DP05_0077PE", "DP05_0077PM", #SOR   
                "DP05_0078PE", "DP05_0078PM", #Multi   
                "DP05_0066PE", "DP05_0066PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_us(year = "2015", dataset = "acs/acs1/profile?", geo_for = "us:*", area_type="national", geo_id = "us", get_vars =  
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0017E", "DP05_0017M", #median age
                "DP05_0072PE", "DP05_0072PM", #white
                "DP05_0073PE", "DP05_0073PM", #black
                "DP05_0074PE", "DP05_0074PM", #AIAN
                "DP05_0075PE", "DP05_0075PM", #Asian
                "DP05_0076PE", "DP05_0076PM", #NHPI    
                "DP05_0077PE", "DP05_0077PM", #SOR   
                "DP05_0078PE", "DP05_0078PM", #Multi   
                "DP05_0066PE", "DP05_0066PM", #Hispanic   
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [4]:
#This is the function for the region and division

def api_rd(year, dataset, geo_for, area_type, geo_id, get_vars):
    global general_measures

    # Build base URL
    HOST = "https://api.census.gov/data"
    base_url = "/".join([HOST, year, dataset])

    # Specify Census variables and other predicates
    predicates = {}
    predicates["get"] = ",".join(get_vars)
    predicates["for"] = geo_for

    # Execute the request, examine text of response object
    r = requests.get(base_url, params=predicates)

    # Construct the DataFrame
    col_names = base_col_names + ['GEO_ID','fips']
    df = pd.DataFrame(columns = col_names, data = r.json()[1:])
                
    # Create additional variables for file
    df["year"] = year
    df["area_type"] = area_type
    df["state_fips"] = ""
    df["county_fips"] = ""
    df["tract_fips"] = ""
    df["GEO_ID"] = df[geo_id]  
    df.drop('fips', axis=1, inplace=True)
   
    #append data
    general_measures = pd.concat([general_measures,df], ignore_index=True)

In [5]:
#region
api_rd(year = "2022", dataset = "acs/acs1/profile?", geo_for = "region:*", area_type="region", geo_id = ["fips"], get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0079PE", "DP05_0079PM", #white
                "DP05_0080PE", "DP05_0080PM", #black
                "DP05_0081PE", "DP05_0081PM", #AIAN
                "DP05_0082PE", "DP05_0082PM", #Asian
                "DP05_0083PE", "DP05_0083PM", #NHPI    
                "DP05_0084PE", "DP05_0084PM", #SOR   
                "DP05_0085PE", "DP05_0085PM", #Multi   
                "DP05_0073PE", "DP05_0073PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_rd(year = "2021", dataset = "acs/acs1/profile?", geo_for = "region:*", area_type="region", geo_id = ["fips"], get_vars =  
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

#1-year data are unavailable for 2020
api_rd(year = "2020", dataset = "acs/acs5/profile?", geo_for = "region:*", area_type="region", geo_id = ["fips"], get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_rd(year = "2019", dataset = "acs/acs1/profile?", geo_for = "region:*", area_type="region", geo_id = ["fips"], get_vars =   
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0114PE", "DP02_0114PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_rd(year = "2018", dataset = "acs/acs1/profile?", geo_for = "region:*", area_type="region", geo_id = ["fips"], get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_rd(year = "2017", dataset = "acs/acs1/profile?", geo_for = "region:*", area_type="region", geo_id = ["fips"], get_vars =   
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_rd(year = "2016", dataset = "acs/acs1/profile?", geo_for = "region:*", area_type="region", geo_id = ["fips"], get_vars =   
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0017E", "DP05_0017M", #median age
                "DP05_0072PE", "DP05_0072PM", #white
                "DP05_0073PE", "DP05_0073PM", #black
                "DP05_0074PE", "DP05_0074PM", #AIAN
                "DP05_0075PE", "DP05_0075PM", #Asian
                "DP05_0076PE", "DP05_0076PM", #NHPI    
                "DP05_0077PE", "DP05_0077PM", #SOR   
                "DP05_0078PE", "DP05_0078PM", #Multi   
                "DP05_0066PE", "DP05_0066PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_rd(year = "2015", dataset = "acs/acs1/profile?", geo_for = "region:*", area_type="region", geo_id = ["fips"], get_vars =   
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0017E", "DP05_0017M", #median age
                "DP05_0072PE", "DP05_0072PM", #white
                "DP05_0073PE", "DP05_0073PM", #black
                "DP05_0074PE", "DP05_0074PM", #AIAN
                "DP05_0075PE", "DP05_0075PM", #Asian
                "DP05_0076PE", "DP05_0076PM", #NHPI    
                "DP05_0077PE", "DP05_0077PM", #SOR   
                "DP05_0078PE", "DP05_0078PM", #Multi   
                "DP05_0066PE", "DP05_0066PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])


In [6]:
#division
api_rd(year = "2022", dataset = "acs/acs1/profile?", geo_for = "division:*", area_type="division", geo_id = ["fips"], get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0079PE", "DP05_0079PM", #white
                "DP05_0080PE", "DP05_0080PM", #black
                "DP05_0081PE", "DP05_0081PM", #AIAN
                "DP05_0082PE", "DP05_0082PM", #Asian
                "DP05_0083PE", "DP05_0083PM", #NHPI    
                "DP05_0084PE", "DP05_0084PM", #SOR   
                "DP05_0085PE", "DP05_0085PM", #Multi   
                "DP05_0073PE", "DP05_0073PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_rd(year = "2021", dataset = "acs/acs1/profile?", geo_for = "division:*", area_type="division", geo_id = ["fips"], get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

#1-year data are unavailable for 2020
api_rd(year = "2020", dataset = "acs/acs5/profile?", geo_for = "division:*", area_type="division", geo_id = ["fips"], get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_rd(year = "2019", dataset = "acs/acs1/profile?", geo_for = "division:*", area_type="division", geo_id = ["fips"], get_vars =  
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0114PE", "DP02_0114PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_rd(year = "2018", dataset = "acs/acs1/profile?", geo_for = "division:*", area_type="division", geo_id = ["fips"], get_vars =  
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_rd(year = "2017", dataset = "acs/acs1/profile?", geo_for = "division:*", area_type="division", geo_id = ["fips"], get_vars =   
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_rd(year = "2016", dataset = "acs/acs1/profile?", geo_for = "division:*", area_type="division", geo_id = ["fips"], get_vars =   
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0017E", "DP05_0017M", #median age
                "DP05_0072PE", "DP05_0072PM", #white
                "DP05_0073PE", "DP05_0073PM", #black
                "DP05_0074PE", "DP05_0074PM", #AIAN
                "DP05_0075PE", "DP05_0075PM", #Asian
                "DP05_0076PE", "DP05_0076PM", #NHPI    
                "DP05_0077PE", "DP05_0077PM", #SOR   
                "DP05_0078PE", "DP05_0078PM", #Multi   
                "DP05_0066PE", "DP05_0066PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

api_rd(year = "2015", dataset = "acs/acs1/profile?", geo_for = "division:*", area_type="division", geo_id = ["fips"], get_vars =   
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0017E", "DP05_0017M", #median age
                "DP05_0072PE", "DP05_0072PM", #white
                "DP05_0073PE", "DP05_0073PM", #black
                "DP05_0074PE", "DP05_0074PM", #AIAN
                "DP05_0075PE", "DP05_0075PM", #Asian
                "DP05_0076PE", "DP05_0076PM", #NHPI    
                "DP05_0077PE", "DP05_0077PM", #SOR   
                "DP05_0078PE", "DP05_0078PM", #Multi   
                "DP05_0066PE", "DP05_0066PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])


In [7]:
#This is the function for the states

def api_st(year, dataset, get_vars):
    global general_measures

    # Build base URL
    HOST = "https://api.census.gov/data"
    base_url = "/".join([HOST, year, dataset])

    # Specify Census variables and other predicates
    predicates = {}
    predicates["get"] = ",".join(get_vars)
    predicates["for"] = "state:*"

    # Execute the request, examine text of response object
    r = requests.get(base_url, params=predicates)

    # Construct the DataFrame
    col_names = base_col_names + ['fips', 'GEO_ID']
    df = pd.DataFrame(columns = col_names, data = r.json()[1:])
                
    # Create additional variables for file
    df["year"] = year
    df["area_type"] = "state"
    df["state_fips"] = df['fips'].str[9:11]
    df["county_fips"] = ""
    df["tract_fips"] = ""
    df["GEO_ID"] = df['fips'].str[9:11]
    df.drop('fips', axis=1, inplace=True)
    
    #append data
    general_measures = pd.concat([general_measures,df], ignore_index=True)
   

In [8]:
api_st(year = "2022", dataset = "acs/acs1/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0079PE", "DP05_0079PM", #white
                "DP05_0080PE", "DP05_0080PM", #black
                "DP05_0081PE", "DP05_0081PM", #AIAN
                "DP05_0082PE", "DP05_0082PM", #Asian
                "DP05_0083PE", "DP05_0083PM", #NHPI    
                "DP05_0084PE", "DP05_0084PM", #SOR   
                "DP05_0085PE", "DP05_0085PM", #Multi   
                "DP05_0073PE", "DP05_0073PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [9]:
api_st(year = "2021", dataset = "acs/acs1/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [10]:
api_st(year = "2020", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [11]:
api_st(year = "2019", dataset = "acs/acs1/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0114PE", "DP02_0114PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [12]:
api_st(year = "2018", dataset = "acs/acs1/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [13]:
api_st(year = "2017", dataset = "acs/acs1/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [14]:
api_st(year = "2016", dataset = "acs/acs1/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0017E", "DP05_0017M", #median age
                "DP05_0072PE", "DP05_0072PM", #white
                "DP05_0073PE", "DP05_0073PM", #black
                "DP05_0074PE", "DP05_0074PM", #AIAN
                "DP05_0075PE", "DP05_0075PM", #Asian
                "DP05_0076PE", "DP05_0076PM", #NHPI    
                "DP05_0077PE", "DP05_0077PM", #SOR   
                "DP05_0078PE", "DP05_0078PM", #Multi   
                "DP05_0066PE", "DP05_0066PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [15]:
api_st(year = "2015", dataset = "acs/acs1/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0017E", "DP05_0017M", #median age
                "DP05_0072PE", "DP05_0072PM", #white
                "DP05_0073PE", "DP05_0073PM", #black
                "DP05_0074PE", "DP05_0074PM", #AIAN
                "DP05_0075PE", "DP05_0075PM", #Asian
                "DP05_0076PE", "DP05_0076PM", #NHPI    
                "DP05_0077PE", "DP05_0077PM", #SOR   
                "DP05_0078PE", "DP05_0078PM", #Multi   
                "DP05_0066PE", "DP05_0066PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [16]:
#This is the function for counties

def api_cty(year, dataset, get_vars):
    global general_measures

    # Build base URL
    HOST = "https://api.census.gov/data"
    base_url = "/".join([HOST, year, dataset])

    # Specify Census variables and other predicates
    predicates = {}
    predicates["get"] = ",".join(get_vars)
    predicates["for"] = "county:*"

    # Execute the request
    r = requests.get(base_url, params=predicates)

    # Construct the DataFrame
    col_names = base_col_names + ['GEO_ID','state_fips', 'county_fips']
    df = pd.DataFrame(columns = col_names, data = r.json()[1:])
                
    # Create additional variables for file
    df["year"] = year
    df["area_type"] = "county"
    df["tract_fips"] = ""
    df["GEO_ID"] = df['GEO_ID'].str[9:14]
   
    #append data
    general_measures = pd.concat([general_measures,df], ignore_index=True)
   

In [17]:
api_cty(year = "2022", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0079PE", "DP05_0079PM", #white
                "DP05_0080PE", "DP05_0080PM", #black
                "DP05_0081PE", "DP05_0081PM", #AIAN
                "DP05_0082PE", "DP05_0082PM", #Asian
                "DP05_0083PE", "DP05_0083PM", #NHPI    
                "DP05_0084PE", "DP05_0084PM", #SOR   
                "DP05_0085PE", "DP05_0085PM", #Multi   
                "DP05_0073PE", "DP05_0073PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [18]:
api_cty(year = "2021", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [19]:
api_cty(year = "2020", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [20]:
api_cty(year = "2019", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0114PE", "DP02_0114PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [21]:
api_cty(year = "2018", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [22]:
api_cty(year = "2017", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [23]:
api_cty(year = "2016", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0017E", "DP05_0017M", #median age
                "DP05_0072PE", "DP05_0072PM", #white
                "DP05_0073PE", "DP05_0073PM", #black
                "DP05_0074PE", "DP05_0074PM", #AIAN
                "DP05_0075PE", "DP05_0075PM", #Asian
                "DP05_0076PE", "DP05_0076PM", #NHPI    
                "DP05_0077PE", "DP05_0077PM", #SOR   
                "DP05_0078PE", "DP05_0078PM", #Multi   
                "DP05_0066PE", "DP05_0066PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [24]:
api_cty(year = "2015", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0017E", "DP05_0017M", #median age
                "DP05_0072PE", "DP05_0072PM", #white
                "DP05_0073PE", "DP05_0073PM", #black
                "DP05_0074PE", "DP05_0074PM", #AIAN
                "DP05_0075PE", "DP05_0075PM", #Asian
                "DP05_0076PE", "DP05_0076PM", #NHPI    
                "DP05_0077PE", "DP05_0077PM", #SOR   
                "DP05_0078PE", "DP05_0078PM", #Multi   
                "DP05_0066PE", "DP05_0066PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [25]:
#This is the function for tribal areas (2019 - 2022)

def api_aian(year, dataset, get_vars):
    global general_measures

    # Build base URL
    HOST = "https://api.census.gov/data"
    base_url = "/".join([HOST, year, dataset])

    # Specify Census variables and other predicates
    predicates = {}
    predicates["get"] = ",".join(get_vars)
    predicates["for"] = "american indian area/alaska native area (reservation or statistical entity only):*"

    # Execute the request, examine text of response object
    r = requests.get(base_url, params=predicates)

    # Construct the DataFrame
    col_names = base_col_names + ['fips', 'GEO_ID']
    df = pd.DataFrame(columns = col_names, data = r.json()[1:])
                
    # Create additional variables for file
    df["year"] = year
    df["area_type"] = "tribal_area"
    df["state_fips"] = ""
    df["county_fips"] = ""
    df["tract_fips"] = ""
    df["GEO_ID"] = df['fips'].str[9:14]
    df.drop('fips', axis=1, inplace=True)
 
    #append data
    general_measures = pd.concat([general_measures,df], ignore_index=True)
   

In [26]:
api_aian(year = "2022", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0079PE", "DP05_0079PM", #white
                "DP05_0080PE", "DP05_0080PM", #black
                "DP05_0081PE", "DP05_0081PM", #AIAN
                "DP05_0082PE", "DP05_0082PM", #Asian
                "DP05_0083PE", "DP05_0083PM", #NHPI    
                "DP05_0084PE", "DP05_0084PM", #SOR   
                "DP05_0085PE", "DP05_0085PM", #Multi   
                "DP05_0073PE", "DP05_0073PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [27]:
api_aian(year = "2021", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [28]:
api_aian(year = "2020", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [29]:
api_aian(year = "2019", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0114PE", "DP02_0114PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [30]:
#This is the function for tribal areas, (2015 - 2018)

def api_aian(year, dataset, get_vars):
    global general_measures

    # Build base URL
    HOST = "https://api.census.gov/data"
    base_url = "/".join([HOST, year, dataset])

    # Specify Census variables and other predicates
    predicates = {}
    predicates["get"] = ",".join(get_vars)
    predicates["for"] = "american indian area/alaska native area (reservation or statistical entity only):*"

    # Execute the request, examine text of response object
    r = requests.get(base_url, params=predicates)

    # Construct the DataFrame
    col_names = base_col_names + ['fips', 'GEO_ID','R']
    df = pd.DataFrame(columns = col_names, data = r.json()[1:])
                
    # Create additional variables for file
    df["year"] = year
    df["area_type"] = "tribal_area"
    df["state_fips"] = ""
    df["county_fips"] = ""
    df["tract_fips"] = ""
    df["GEO_ID"] = df['fips'].str[9:14]
    df.drop('fips', axis=1, inplace=True)
    df.drop('R', axis=1, inplace=True)
        
    #append data
    general_measures = pd.concat([general_measures,df], ignore_index=True)
   

In [31]:
#The file changes when you get to year 2018

api_aian(year = "2018", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [32]:
api_aian(year = "2017", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [33]:
api_aian(year = "2016", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0017E", "DP05_0017M", #median age
                "DP05_0072PE", "DP05_0072PM", #white
                "DP05_0073PE", "DP05_0073PM", #black
                "DP05_0074PE", "DP05_0074PM", #AIAN
                "DP05_0075PE", "DP05_0075PM", #Asian
                "DP05_0076PE", "DP05_0076PM", #NHPI    
                "DP05_0077PE", "DP05_0077PM", #SOR   
                "DP05_0078PE", "DP05_0078PM", #Multi   
                "DP05_0066PE", "DP05_0066PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [34]:
api_aian(year = "2015", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0017E", "DP05_0017M", #median age
                "DP05_0072PE", "DP05_0072PM", #white
                "DP05_0073PE", "DP05_0073PM", #black
                "DP05_0074PE", "DP05_0074PM", #AIAN
                "DP05_0075PE", "DP05_0075PM", #Asian
                "DP05_0076PE", "DP05_0076PM", #NHPI    
                "DP05_0077PE", "DP05_0077PM", #SOR   
                "DP05_0078PE", "DP05_0078PM", #Multi   
                "DP05_0066PE", "DP05_0066PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [35]:
#This is the function for the counties

def api_tract(year, dataset, get_vars):
    global general_measures

    # Build base URL
    HOST = "https://api.census.gov/data"
    base_url = "/".join([HOST, year, dataset])

    # Specify Census variables and other predicates
    predicates = {}
    predicates["get"] = ",".join(get_vars)
    predicates["for"] = "tract:*"
    predicates["in"] = "state:01,02,04,05,06,08,09,10,11,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,48,49,50,51,53,54,55,56,72"

    # Execute the request, examine text of response object
    r = requests.get(base_url, params=predicates)

    # Construct the DataFrame
    col_names = base_col_names + ['GEO_ID','state_fips', 'county_fips', 'tract_fips']
    df = pd.DataFrame(columns = col_names, data = r.json()[1:])
                
    # Create additional variables for file
    df["year"] = year
    df["area_type"] = "tract"
    df["GEO_ID"] = df['GEO_ID'].str[9:20]
       
    #append data
    general_measures = pd.concat([general_measures,df], ignore_index=True)
   

In [36]:
api_tract(year = "2022", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0079PE", "DP05_0079PM", #white
                "DP05_0080PE", "DP05_0080PM", #black
                "DP05_0081PE", "DP05_0081PM", #AIAN
                "DP05_0082PE", "DP05_0082PM", #Asian
                "DP05_0083PE", "DP05_0083PM", #NHPI    
                "DP05_0084PE", "DP05_0084PM", #SOR   
                "DP05_0085PE", "DP05_0085PM", #Multi   
                "DP05_0073PE", "DP05_0073PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [37]:
api_tract(year = "2021", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [38]:
api_tract(year = "2020", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0115PE", "DP02_0115PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [39]:
api_tract(year = "2019", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0068PE", "DP02_0068PM", #bachelor
                "DP02_0067PE", "DP02_0067PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0114PE", "DP02_0114PM", #language
                "DP02_0070PE", "DP02_0070PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [40]:
api_tract(year = "2018", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [41]:
api_tract(year = "2017", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0018E", "DP05_0018M", #median age
                "DP05_0077PE", "DP05_0077PM", #white
                "DP05_0078PE", "DP05_0078PM", #black
                "DP05_0079PE", "DP05_0079PM", #AIAN
                "DP05_0080PE", "DP05_0080PM", #Asian
                "DP05_0081PE", "DP05_0081PM", #NHPI    
                "DP05_0082PE", "DP05_0082PM", #SOR   
                "DP05_0083PE", "DP05_0083PM", #Multi   
                "DP05_0071PE", "DP05_0071PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [42]:
api_tract(year = "2016", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0017E", "DP05_0017M", #median age
                "DP05_0072PE", "DP05_0072PM", #white
                "DP05_0073PE", "DP05_0073PM", #black
                "DP05_0074PE", "DP05_0074PM", #AIAN
                "DP05_0075PE", "DP05_0075PM", #Asian
                "DP05_0076PE", "DP05_0076PM", #NHPI    
                "DP05_0077PE", "DP05_0077PM", #SOR   
                "DP05_0078PE", "DP05_0078PM", #Multi   
                "DP05_0066PE", "DP05_0066PM", #Hispanic   
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [43]:
api_tract(year = "2015", dataset = "acs/acs5/profile?", get_vars = 
       ["NAME", "DP05_0001E", "DP05_0001M", #total pop
                "DP05_0017E", "DP05_0017M", #median age
                "DP05_0072PE", "DP05_0072PM", #white
                "DP05_0073PE", "DP05_0073PM", #black
                "DP05_0074PE", "DP05_0074PM", #AIAN
                "DP05_0075PE", "DP05_0075PM", #Asian
                "DP05_0076PE", "DP05_0076PM", #NHPI    
                "DP05_0077PE", "DP05_0077PM", #SOR   
                "DP05_0078PE", "DP05_0078PM", #Multi   
                "DP05_0066PE", "DP05_0066PM", #Hispanic  
                "DP02_0067PE", "DP02_0067PM", #bachelor
                "DP02_0066PE", "DP02_0066PM", #highschool
                "DP03_0062E", "DP03_0062M", #income
                "DP03_0128PE", "DP03_0128PM", #poverty
                "DP03_0004PE", "DP03_0004PM", #employed
                "DP03_0005PE", "DP03_0005PM", #unemployed
                "DP02_0113PE", "DP02_0113PM", #language
                "DP02_0069PE", "DP02_0069PM", #veteran
                "DP03_0099PE", "DP03_0099PM", #health
                "GEO_ID"])

In [44]:
# Create CSV
general_measures.to_csv("/data/discover/Data/General/general_measures.csv", header=True, index=False)